In [1]:
#default_exp liftover

In [14]:
#hide
%load_ext autoreload
%autoreload 2

# Liftover module

> Liftover genodata and sumstat

In [2]:
#hide
from nbdev.showdoc import *

In [1]:
#export
import gzip
from liftover import get_lifter

In [54]:
#export
class Liftover:
    def __init__(self,fr='hg19',to='hg38'):
        self.fr,self.to = fr,to
        self.chainmap = get_lifter(fr, to)
        #self.fasta 
              
    def variants_liftover(self,chrom,pos):
        if len(chrom) == 1:
            chrom = [chrom]*len(pos)
        lchr,lpos = [],[]
        for c,p in zip(chrom,pos):
            new_c,new_p = self.chrpos_liftover(c,p)
            lchr.append(new_c)
            lpos.append(new_p)
        return lchr,lpos
    
    def chrpos_liftover(self,chrom,pos):
        try:
            if chrom == 'X':
                chrom = 23
            elif chrom =='Y':
                chrom = 24
            new_c,new_p,_ = self.chainmap[int(chrom)][pos][0]
            return int(new_c[3:]),new_p
        except:
            return 0,0
           
        #The function to liftover bim
    def bim_liftover(self,bim):
        new_bim = bim.copy()
        lchr,lpos = self.variants_liftover(bim.chrom,bim.pos)
        new_bim.chrom =lchr
        new_bim.pos = lpos
        new_bim.snp = 'chr'+new_bim[['chrom','pos','a0','a1']].astype(str).agg(':'.join, axis=1)
        return new_bim

    
    def sumstat_liftover(self,ss):
        new_ss = ss.copy()
        lchr,lpos = self.variants_liftover(ss.CHR,ss.POS)
        new_ss.CHR =lchr
        new_ss.POS = lpos
        new_ss.SNP = 'chr'+new_ss[['CHR','POS','REF','ALT']].astype(str).agg(':'.join, axis=1)
        return new_ss
    
    def vcf_liftover(self,vcf,vcf_out=None):
        if vcf_out is None:
            vcf_out = vcf[:-7]+'_'+self.fr+'To'+self.to+vcf[-7:]
        count_fail,total= 0,0
        with gzip.open(vcf, 'rt') as ifile:
            with gzip.open(vcf_out,'wt') as ofile:
                for line in ifile:
                    if line.startswith("#"):
                        ofile.write(line)
                    else:
                        variant = [x for x in line.split('\t')]
                        new_c,new_p = self.chrpos_liftover(variant[0],int(variant[1]))
                        if new_c == 0:
                            count_fail +=1
                        total +=1
                        variant[0] = str(new_c)
                        variant[1] = str(new_p)
                        variant[2] = 'chr'+':'.join(variant[:2]+variant[3:5])
                        ofile.write('\t'.join(variant))
            ofile.close()       
        ifile.close()
        print("Total number SNPs ",total,". The number of SNPs failed to liftover " count_fail)
        
    def region_liftover(self,region):
        imp_cs,imp_start = self.chrpos_liftover(region[0],region[1])
        imp_ce,imp_end = self.chrpos_liftover(region[0],region[2])
        if imp_cs !=imp_ce:
            raise ValueError('After liftover, the region is not in the same chromosome anymore.')
        return imp_cs,imp_start,imp_end
    
    def df_liftover(self):
        pass


In [55]:
lf = Liftover('hg19','hg38')

In [11]:
vcf ='data/GH.AR.SAD.P1.001.0_X3547_S42_1180478_GVCF.hard-filtered.gvcf.gz'

In [56]:
lf.region_liftover([5,272741,1213528-900000])

(5, 272626, 313413)

In [12]:
lf.vcf_liftover(vcf)

### Test liftover sumstat

In [16]:
from LDtools.sumstat import Sumstat
from LDtools.liftover import Liftover
def gwas_liftover(input_path,output_path,output_unmapped,output_mapped,fr='hg19',to='hg38',remove_missing=False):
    lf = Liftover(fr,to)
    print("reading GWAS sumstat")
    sums = Sumstat(input_path)
    print("liftover from" + fr +"to" +to)
    sums1 = lf.sumstat_liftover(sums.ss)
    if remove_missing:
        sums1[sums1.CHR == 0].to_csv(output_unmapped, compression='gzip', sep = "\t", header = True, index = False)
        sums1[sums1.CHR != 0].to_csv(output_mapped, compression='gzip', sep = "\t", header = True, index = False)
    else:
        sums1.to_csv(output_path, compression='gzip', sep = "\t", header = True, index = False)

In [ ]:
def gwas_liftover(input_file,output_path=None,fr='hg19',to='hg38',remove_missing=False):
    if output_path is None:
        output_path = os.path.dirname(input_file)+'/'
    basename = os.path.basename(input_file)
    lf = Liftover('hg19','hg38')
    print("reading GWAS sumstat")
    sums = Sumstat(input_path)
    print("liftover from" + fr +"to" +to)
    sums1 = lf.sumstat_liftover(sums.ss)
    if remove_missing:
        sums1[sums1.CHR == 0].to_csv(output_unmapped, compression='gzip', sep = "\t", header = True, index = False)
        sums1[sums1.CHR != 0].to_csv(output_mapped, compression='gzip', sep = "\t", header = True, index = False)
    else:
        sums1.to_csv(output_path, compression='gzip', sep = "\t", header = True, index = False)

In [ ]:
sumstats_lifted = f'{cwd}/{_input:bnn}.hg38.sumstats.gz',
sumstats_unmapped = f'{cwd}/{_input:bnn}.hg38.sumstats_unmapped.gz',
sumstats_mapped = f'{cwd}/{_input:bnn}.hg38.sumstats_mapped.gz'

In [37]:
import os

In [41]:
tmp = os.path.basename(input_path)

In [42]:
os.path.splitext(tmp)

('100521_UKBB_Combined_f2247_f2257_expandedwhite_93258cases_237318ctrl_500k_PC1_PC2_f2247_f2257.regenie.snp_stats',
 '.gz')

In [40]:
os.path.dirname(input_path)+'/'

'/home/dmc2245/UKBiobank/results/REGENIE_results/results_imputed_data/2021_10_07_combined_500K/'

In [19]:
sums = Sumstat(input_path)

In [33]:
lf = Liftover('hg19','hg38')

In [34]:
lf.sumstat_liftover(sums.ss[:10])

,CHR,POS,REF,ALT,SNP,BETA,SE,P
0,1,13259,G,A,chr1:13259:G:A,0.434586,0.175780,0.014801
1,1,17569,C,A,chr1:17569:C:A,-0.030568,0.795968,0.969366
2,1,17641,G,A,chr1:17641:G:A,-0.078881,0.108663,0.467883
3,1,30741,C,A,chr1:30741:C:A,-1.599610,0.990472,0.044798
4,1,57222,T,C,chr1:57222:T:C,0.031666,0.121422,0.794253
5,1,58396,T,C,chr1:58396:T:C,0.366266,0.172004,0.035663
6,1,62157,G,A,chr1:62157:G:A,-0.147251,0.296105,0.618983
7,1,62595,C,T,chr1:62595:C:T,0.356993,0.171623,0.040096
8,1,69487,G,A,chr1:69487:G:A,-0.559373,0.853882,0.512407
9,1,69569,T,C,chr1:69569:T:C,0.232690,0.216585,0.282662


In [25]:

def main(input_path,output_path,remove_missing):
    sums = read_regenie(input_path)
    sums1 = sumstat_liftover(sums)
    if remove_missing:
        sums1[sums1.CHR == 0].to_csv(output_path, sep = "\t", header = True, index = False)
    else:
        sums1.to_csv(output_path, sep = "\t", header = True, index = False)

In [18]:
input_path = '/home/dmc2245/UKBiobank/results/REGENIE_results/results_imputed_data/2021_10_07_combined_500K/100521_UKBB_Combined_f2247_f2257_expandedwhite_93258cases_237318ctrl_500k_PC1_PC2_f2247_f2257.regenie.snp_stats.gz'
output_path = ''
remove_missing = True

In [2]:
main(input_path,output_path,remove_missing)

In [44]:
':'.join([1,'1'])

TypeError: sequence item 0: expected str instance, int found